In [ ]:
import pyodbc
import pandas as pd
import numpy as np
import os

In [ ]:
pd.set_option('display.float_format','{:.2f}'.format)

In [ ]:
data = pd.read_parquet('Data/data_gl_id_mapped_2025.parquet')

In [ ]:
data.columns

In [ ]:
unique_df = data[['unique_cust_id','STO_FIN_HRCHY__DCSTAR','month','total_sale_amount','e_sale_amount']]

In [ ]:
unique_df['unique_cust_id'] = unique_df['unique_cust_id'].fillna('None')
unique_df['unique_cust_id'].isna().sum()

In [ ]:
data[data['STO_FIN_HRCHY__DCSTAR']=='ALT4757148'].sort_values(by='month')

In [ ]:
unique_df[unique_df['STO_FIN_HRCHY__DCSTAR']=='ALT4757148']

In [ ]:
# dcstar_to_gid = unique_df.groupby('STO_FIN_HRCHY__DCSTAR')['unique_cust_id'].nunique().reset_index(name='gid_count')
# gid_to_dcstar = unique_df.groupby('unique_cust_id')['STO_FIN_HRCHY__DCSTAR'].nunique().reset_index(name='dcstar_count')

In [ ]:
dcstar_to_gid_rev = unique_df.groupby(['STO_FIN_HRCHY__DCSTAR','month']).agg({
    'unique_cust_id':'nunique',
    'total_sale_amount' : 'sum',
    'e_sale_amount' : 'sum'
}).reset_index()

In [ ]:
list_gid_2 = dcstar_to_gid_rev[(dcstar_to_gid_rev['unique_cust_id']>1)][['STO_FIN_HRCHY__DCSTAR','month','unique_cust_id','total_sale_amount','e_sale_amount']]

In [ ]:
list_gid_2['total_sale_amount_div'] = list_gid_2['total_sale_amount']/list_gid_2['unique_cust_id']
list_gid_2['e_sale_amount_div'] = list_gid_2['e_sale_amount']/list_gid_2['unique_cust_id']

In [ ]:
list_gid_2['total_sale_amount_diff'] = list_gid_2['total_sale_amount'] - list_gid_2['total_sale_amount_div']
list_gid_2['e_sale_amount_diff'] = list_gid_2['e_sale_amount'] - list_gid_2['e_sale_amount_div']

In [ ]:
list_gid_2['total_sale_amount_diff'].sum()

In [ ]:
data.groupby('month')['total_sale_amount'].sum()

In [ ]:
comparision = list_gid_2.groupby('month')['total_sale_amount_diff'].sum().reset_index().merge(data.groupby('month')['total_sale_amount'].sum(),on='month',how='inner')

In [ ]:
comparision['pct_impact'] = (comparision['total_sale_amount_diff']/comparision['total_sale_amount']).apply(lambda x: f'{x: .2%}')
comparision['total_sale_amount_final'] = comparision['total_sale_amount'] - comparision['total_sale_amount_diff']
comparision

In [ ]:
list_gid_2['total_sale_amount_div'].sum()

In [ ]:
data['total_sale_amount'].sum()

In [ ]:
data.columns

In [ ]:
data['global_id'] = np.select([data['unique_cust_id'].isna()],[data['STO_FIN_HRCHY__DCSTAR']],default=data['unique_cust_id'])

In [ ]:
mapping = data.groupby('STO_FIN_HRCHY__DCSTAR',as_index=False)['global_id'].min()

In [ ]:
data.drop(columns='global_id',inplace=True)

In [ ]:
mapping

In [ ]:
data_cleaned = data.merge(mapping,on='STO_FIN_HRCHY__DCSTAR',how='left')

In [ ]:
data_cleaned.drop_duplicates(subset=['STO_FIN_HRCHY__DCSTAR','month'],inplace=True)

In [ ]:
data_cleaned.groupby('month')['total_sale_amount'].sum()

In [ ]:
data_cleaned.groupby('month')['total_sale_amount'].sum()

In [ ]:
data_cleaned.to_parquet('Data/data_gl_id_mapped_2025_cleaned_v2.parquet')